# Game Development in Python

This workshop goes through building a game in Python 3 using the PyGame Library

## How to Setup:

- Clone this repo to work locally
    - This workshop doesn't use Google Colab because Colab environments don't come with GPUs, so we can't run our games.
- Install a way to read Jupyter Notebooks
    - The workshop is offered through a Notebook so that we can break down each part and provide a lot of detail
    - You could [upload](https://jupyter.org/try) this entire repo to JupyterLab to work online, or [install](https://jupyter.org/install) JupyterLab locally.
    - Or, you could open the repo in [VS Code](https://code.visualstudio.com/download) and the Notebook can be viewed there

- Import `pygame` and `random` modules
- Then, import some locals. You can think of these like pygame keywords.
    - `RLEACCEL` means run-length encoding. Basically, this is how we render sprites
    - `K_UP, K_DOWN, K_LEFT, K_RIGHT` are used for different keys within our game. We will use these to move our player. 
    - `KEYDOWN` is used to capture any keypress. This will be used in tandem with `K_ESCAPE` to exit the game.
    - `QUIT` is an event used to quit the game, which will be triggered from the aforementioned.
- `SCREEN_WIDTH` and `SCREEN_HEIGHT` are used to define the window size.

In [ ]:
# Import the pygame module
import pygame

# Import random for random numbers
import random

# Import pygame.locals for easier access to key coordinates
# Updated to conform to flake8 and black standards
# from pygame.locals import *
from pygame.locals import (
    RLEACCEL,
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

# Define constants for the screen width and height
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

We will create a Sprite for our Player. This will be the unit that we control in the game. 

- In the constructor, We define a `surf` and a `rect`. These are used to control the appearance and the position of the sprite accordingly.
- We use an `update` method to define how the sprite will move and interact with the rest of the game. 
    - For each pressed key, move our sprite in the respective direction.
    - If the sprite moves off of the screen, move them back. 

In [ ]:
# Define the Player object extending pygame.sprite.Sprite
# Instead of a surface, we use an image for a better looking sprite
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super(Player, self).__init__()
        self.surf = pygame.image.load("jet.png").convert()
        self.surf.set_colorkey((255, 255, 255), RLEACCEL)
        self.rect = self.surf.get_rect()

    # Move the sprite based on keypresses
    def update(self, pressed_keys):
        if pressed_keys[K_UP]:
            self.rect.move_ip(0, -5)
        if pressed_keys[K_DOWN]:
            self.rect.move_ip(0, 5)
        if pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if pressed_keys[K_RIGHT]:
            self.rect.move_ip(5, 0)

        # Keep player on the screen
        if self.rect.left < 0:
            self.rect.left = 0
        elif self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        elif self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

Next, we define a sprite for the enemies.

- In the constructor, we again use a `surf` and `rect`.
- When we define our object's original position, we can't always use the same starting point for every object or the game wouldn't be challenging.
    - We use random values within a range to define the starting point.
    - For the horizontal, we wouldn't want to spawn objects on screen, we would want to spawn objects off of the right side and have them fly into view. So, we use a range that starts larger than `SCREEN_WIDTH`
    - Enemies should spawn at any height, so we use a range of 0 to `SCREEN_HEIGHT`
- The velocity of the enemies should vary. So, we use a random `self.speed` to define the sprite's speed per sprite.
- In the `update()`, This is where we use our `self.speed`.
    - Every update, we move our object to the left by `self.speed`.
    - When the object goes off of the left side of the screen, we can `kill` it.

In [ ]:
# Define the enemy object extending pygame.sprite.Sprite
# Instead of a surface, we use an image for a better looking sprite
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super(Enemy, self).__init__()
        self.surf = pygame.image.load("missile.png").convert()
        self.surf.set_colorkey((255, 255, 255), RLEACCEL)
        # The starting position is randomly generated, as is the speed
        self.rect = self.surf.get_rect(
            center=(
                random.randint(SCREEN_WIDTH + 20, SCREEN_WIDTH + 100),
                random.randint(0, SCREEN_HEIGHT),
            )
        )
        self.speed = random.randint(5, 20)

    # Move the enemy based on speed
    # Remove it when it passes the left edge of the screen
    def update(self):
        self.rect.move_ip(-self.speed, 0)
        if self.rect.right < 0:
            self.kill()

Lastly, a cloud sprite.

- We define the `surf` and `rect` in almost the exact same way that we do our `Enemy`s. 
- The difference lies in how we update the sprite. Instead, all the `Cloud`s should move at the same speed so we don't need a random value, we simply move the cloud over by -5 and `kill` when they leave the screen.

In [ ]:
# Define the cloud object extending pygame.sprite.Sprite
# Use an image for a better looking sprite
class Cloud(pygame.sprite.Sprite):
    def __init__(self):
        super(Cloud, self).__init__()
        self.surf = pygame.image.load("cloud.png").convert()
        self.surf.set_colorkey((0, 0, 0), RLEACCEL)
        # The starting position is randomly generated
        self.rect = self.surf.get_rect(
            center=(
                random.randint(SCREEN_WIDTH + 20, SCREEN_WIDTH + 100),
                random.randint(0, SCREEN_HEIGHT),
            )
        )

    # Move the cloud based on a constant speed
    # Remove it when it passes the left edge of the screen
    def update(self):
        self.rect.move_ip(-5, 0)
        if self.rect.right < 0:
            self.kill()

- `pygame.init()` will let us start an instance of our game.
- the `Clock` object is used to keep framerate of the game.
- we use `pygame.display.set_mode()` to set the size of our screen to our predefined constants.
- We create two events for adding clouds and enemies respectively. pygame has 32 event slots, and 23 are already used. So, we use the constant `pygame.USEREVENT` and then add to it to create unique event IDs
    - for enemies, we define a value of `pygame.USEREVENT + 1` called `ADDENEMY`. 
    - Then, we create a timer with `set_timer` where for every 250 ticks, we trigged the event of the ID that is `ADDENEMY`. 
    - The result is, every 250 ticks we are calling this event. 
    - The *behavior* of the event is defined separately. For now, we just say how often the event happens.
    - The same procedure is done for `ADDCLOUD` except every 1000 ticks.
- We create an instance of `Player`
- Since many groups of sprites share similar behaviors, we need to create groups for them.
- `Enemy`s and `Cloud`s each get their own group named respectively. We also create a group for `all_sprites`.

In [ ]:
# Initialize pygame
pygame.init()

# Setup the clock for a decent framerate
clock = pygame.time.Clock()

# Create the screen object
# The size is determined by the constant SCREEN_WIDTH and SCREEN_HEIGHT
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create custom events for adding a new enemy and cloud
ADDENEMY = pygame.USEREVENT + 1
pygame.time.set_timer(ADDENEMY, 250)
ADDCLOUD = pygame.USEREVENT + 2
pygame.time.set_timer(ADDCLOUD, 1000)

# Create our 'player'
player = Player()

# Create groups to hold enemy sprites, cloud sprites, and all sprites
# - enemies is used for collision detection and position updates
# - clouds is used for position updates
# - all_sprites isused for rendering
enemies = pygame.sprite.Group()
clouds = pygame.sprite.Group()
all_sprites = pygame.sprite.Group()
all_sprites.add(player)

The game's event loop is a central part of the game's logic that controls the flow of the game and handles user input. It repeatedly checks for events (such as keyboard inputs, mouse movements, or window close events) and updates the game state accordingly. The event loop contains the following steps:

- Handle events: The event loop uses the `pygame.event.get()` function to retrieve a list of all the events that have occurred since the last iteration of the loop. It then processes each event in the list using a series of if statements, checking for specific events such as `KEYDOWN` or `QUIT`.
    - On `KEYDOWN`, if the key was `K_ESCAPE,` we flip `running` to `False` and the game ends.
    - The same thing happens if the user closes the game window (x in the top left/right corner)
    - For the `ADDENEMY` or `ADDCLOUD` events, we run the constructor which adds these into the game, then add the sprites into their respective groups. 
- Update game state: The game state is updated based on the events that have been processed in the previous step. This could involve moving the player sprite, generating new enemies or clouds, or updating scores.
    - We run the `update` function of all the sprites in the game.
- Render the game: The updated game state is then rendered on the screen using the pygame.display.update() function.
    - We fill the screen with a light blue
    - Then, loop through `all_sprites` and place each one in the game..
    - Below the player lose check, use `pygame.display.flip()` to update the screen with our changes.
    - `clock.tick` is used to keep the game at 30 frames-per-second.
- Check if the player loses: If the player has collided with any enemy, kill the player and change `running` to `False`.
- Repeat: The loop continues to repeat until the game is quit, either by the player pressing the escape key or by closing the window.

In [ ]:
# Our main loop
while running:
    # Look at every event in the queue
    for event in pygame.event.get():
        # Did the user hit a key?
        if event.type == KEYDOWN:
            # Was it the Escape key? If so, stop the loop
            if event.key == K_ESCAPE:
                running = False

        # Did the user click the window close button? If so, stop the loop
        elif event.type == QUIT:
            running = False

        # Should we add a new enemy?
        elif event.type == ADDENEMY:
            # Create the new enemy, and add it to our sprite groups
            new_enemy = Enemy()
            enemies.add(new_enemy)
            all_sprites.add(new_enemy)

        # Should we add a new cloud?
        elif event.type == ADDCLOUD:
            # Create the new cloud, and add it to our sprite groups
            new_cloud = Cloud()
            clouds.add(new_cloud)
            all_sprites.add(new_cloud)

    # Get the set of keys pressed and check for user input
    pressed_keys = pygame.key.get_pressed()
    player.update(pressed_keys)

    # Update the position of our enemies and clouds
    enemies.update()
    clouds.update()

    # Fill the screen with sky blue
    screen.fill((135, 206, 250))

    # Draw all our sprites
    for entity in all_sprites:
        screen.blit(entity.surf, entity.rect)

    # Check if any enemies have collided with the player
    if pygame.sprite.spritecollideany(player, enemies):
        # If so, remove the player
        player.kill()

        # Stop the loop
        running = False

    # Flip everything to the display
    pygame.display.flip()

    # Ensure we maintain a 30 frames per second rate
    clock.tick(30)


# Final Code

In [1]:
# Import the pygame module
import pygame

# Import random for random numbers
import random

# Import pygame.locals for easier access to key coordinates
# Updated to conform to flake8 and black standards
# from pygame.locals import *
from pygame.locals import (
    RLEACCEL,
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

# Define constants for the screen width and height
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# Define the Player object extending pygame.sprite.Sprite
# Instead of a surface, we use an image for a better looking sprite
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super(Player, self).__init__()
        self.surf = pygame.image.load("jet.png").convert()
        self.surf.set_colorkey((255, 255, 255), RLEACCEL)
        self.rect = self.surf.get_rect()

    # Move the sprite based on keypresses
    def update(self, pressed_keys):
        if pressed_keys[K_UP]:
            self.rect.move_ip(0, -5)
        if pressed_keys[K_DOWN]:
            self.rect.move_ip(0, 5)
        if pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if pressed_keys[K_RIGHT]:
            self.rect.move_ip(5, 0)

        # Keep player on the screen
        if self.rect.left < 0:
            self.rect.left = 0
        elif self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        elif self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT


# Define the enemy object extending pygame.sprite.Sprite
# Instead of a surface, we use an image for a better looking sprite
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super(Enemy, self).__init__()
        self.surf = pygame.image.load("missile.png").convert()
        self.surf.set_colorkey((255, 255, 255), RLEACCEL)
        # The starting position is randomly generated, as is the speed
        self.rect = self.surf.get_rect(
            center=(
                random.randint(SCREEN_WIDTH + 20, SCREEN_WIDTH + 100),
                random.randint(0, SCREEN_HEIGHT),
            )
        )
        self.speed = random.randint(5, 20)

    # Move the enemy based on speed
    # Remove it when it passes the left edge of the screen
    def update(self):
        self.rect.move_ip(-self.speed, 0)
        if self.rect.right < 0:
            self.kill()


# Define the cloud object extending pygame.sprite.Sprite
# Use an image for a better looking sprite
class Cloud(pygame.sprite.Sprite):
    def __init__(self):
        super(Cloud, self).__init__()
        self.surf = pygame.image.load("cloud.png").convert()
        self.surf.set_colorkey((0, 0, 0), RLEACCEL)
        # The starting position is randomly generated
        self.rect = self.surf.get_rect(
            center=(
                random.randint(SCREEN_WIDTH + 20, SCREEN_WIDTH + 100),
                random.randint(0, SCREEN_HEIGHT),
            )
        )

    # Move the cloud based on a constant speed
    # Remove it when it passes the left edge of the screen
    def update(self):
        self.rect.move_ip(-5, 0)
        if self.rect.right < 0:
            self.kill()

# Initialize pygame
pygame.init()

# Setup the clock for a decent framerate
clock = pygame.time.Clock()

# Create the screen object
# The size is determined by the constant SCREEN_WIDTH and SCREEN_HEIGHT
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create custom events for adding a new enemy and cloud
ADDENEMY = pygame.USEREVENT + 1
pygame.time.set_timer(ADDENEMY, 250)
ADDCLOUD = pygame.USEREVENT + 2
pygame.time.set_timer(ADDCLOUD, 1000)

# Create our 'player'
player = Player()

# Create groups to hold enemy sprites, cloud sprites, and all sprites
# - enemies is used for collision detection and position updates
# - clouds is used for position updates
# - all_sprites isused for rendering
enemies = pygame.sprite.Group()
clouds = pygame.sprite.Group()
all_sprites = pygame.sprite.Group()
all_sprites.add(player)

# Variable to keep our main loop running
running = True

# Our main loop
while running:
    # Look at every event in the queue
    for event in pygame.event.get():
        # Did the user hit a key?
        if event.type == KEYDOWN:
            # Was it the Escape key? If so, stop the loop
            if event.key == K_ESCAPE:
                running = False

        # Did the user click the window close button? If so, stop the loop
        elif event.type == QUIT:
            running = False

        # Should we add a new enemy?
        elif event.type == ADDENEMY:
            # Create the new enemy, and add it to our sprite groups
            new_enemy = Enemy()
            enemies.add(new_enemy)
            all_sprites.add(new_enemy)

        # Should we add a new cloud?
        elif event.type == ADDCLOUD:
            # Create the new cloud, and add it to our sprite groups
            new_cloud = Cloud()
            clouds.add(new_cloud)
            all_sprites.add(new_cloud)

    # Get the set of keys pressed and check for user input
    pressed_keys = pygame.key.get_pressed()
    player.update(pressed_keys)

    # Update the position of our enemies and clouds
    enemies.update()
    clouds.update()

    # Fill the screen with sky blue
    screen.fill((135, 206, 250))

    # Draw all our sprites
    for entity in all_sprites:
        screen.blit(entity.surf, entity.rect)

    # Check if any enemies have collided with the player
    if pygame.sprite.spritecollideany(player, enemies):
        # If so, remove the player
        player.kill()

        # Stop the loop
        running = False

    # Flip everything to the display
    pygame.display.flip()

    # Ensure we maintain a 30 frames per second rate
    clock.tick(30)


pygame 2.1.3 (SDL 2.0.22, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 